In [14]:
import pandas as pd

df = pd.read_csv(r'gen_dataset.csv')
print(df)
text_data = list(df["text"])
class_data = df["class"].to_numpy()
for i in range(len(class_data)):
    if class_data[i]=='notsarc':
        class_data[i]=0
    else:
        class_data[i]=1

        class    id                                               text
0     notsarc     1  If that's true, then Freedom of Speech is doom...
1     notsarc     2  Neener neener - is it time to go in from the p...
2     notsarc     3  Just like the plastic gun fear, the armour pie...
3     notsarc     4  So geology is a religion because we weren't he...
4     notsarc     5  Well done Monty. Mark that up as your first ev...
...       ...   ...                                                ...
6515     sarc  6516  depends on when the baby bird died.   run alon...
6516     sarc  6517  ok, sheesh, to clarify, women who arent aborti...
6517     sarc  6518  so..   eh??   hows this sound?   will it fly w...
6518     sarc  6519  I think we should put to a vote, the right of ...
6519     sarc  6520  You have a blob of tissue in your "but(sic)"? ...

[6520 rows x 3 columns]


In [25]:
df = pd.read_csv(r'imdb_dataset.csv')
print(df)
text_data1 = list(df["review"])
class_data1 = df["sentiment"].to_numpy()
for i in range(len(class_data1)):
    if class_data1[i]=='negative':
        class_data1[i]=1
    else:
        class_data1[i]=0

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [26]:
class_data = list(class_data)
class_data1

array([0, 0, 0, ..., 1, 1, 1], dtype=object)

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Load the pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the classification model
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        logits = self.fc(pooled_output)
        return logits

# Load the training data
# train_texts = ["Example sentence 1", "Example sentence 2", ...]
# train_labels = [0, 1, ...]

# Tokenize the text data
train_encodings = tokenizer(list(text_data1), padding=True, truncation=True, max_length=128)

# Convert the data into PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(list(class_data1))
)

# Define the training parameters
classifier = BertClassifier(model, num_classes=2)
classifier.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=2e-5)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Train the BERT classifier
classifier.train()
for epoch in range(10):
    print("Epoch ", epoch+1)
    losses = []
    for idx, batch in enumerate(train_loader):
        # batch.to(device)
        if(idx % 20 == 0):
            print("-", end="")
        optimizer.zero_grad()
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        loss.backward()
        losses.append(loss.cpu().detach().numpy())
        optimizer.step()
    print()
    print(sum(losses) / len(losses))

cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch  1
-------------------------------------------------------------------------------
0.29737370189276935
Epoch  2
-------------------------------------------------------------------------------
0.18280126903175126
Epoch  3
-------------------------------------------------------------------------------
0.09722322550848346
Epoch  4
-------------------------------------------------------------------------------
0.052874713493046316
Epoch  5
-------------------------------------------------------------------------------
0.03429361733124411
Epoch  6
-------------------------------------------------------------------------------
0.02507596433852802
Epoch  7
-------------------------------------------------------------------------------
0.021117356427392278
Epoch  8
-------------------------------------------------------------------------------
0.019125640004978593
Epoch  9
-------------------------------------------------------------------------------
0.015270659581770315
Epoch  10
-----

In [28]:




# Load the training data
# train_texts = ["Example sentence 1", "Example sentence 2", ...]
# train_labels = [0, 1, ...]

# Tokenize the text data
train_encodings = tokenizer(list(text_data), padding=True, truncation=True, max_length=128)

# Convert the data into PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(list(class_data))
)

# Define the training parameters
# classifier = BertClassifier(model, num_classes=2)
# classifier.to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(classifier.parameters(), lr=2e-5)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)

# Train the BERT classifier
classifier.train()
for epoch in range(10):
    print("Epoch ", epoch+1)
    losses = []
    for idx, batch in enumerate(train_loader):
        # batch.to(device)
        if(idx % 20 == 0):
            print("-", end="")
        optimizer.zero_grad()
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        loss.backward()
        losses.append(loss.cpu().detach().numpy())
        optimizer.step()
    print()
    print(sum(losses) / len(losses))

Epoch  1
----------------------------------------------------------------------------------
0.07238836074604751
Epoch  2
----------------------------------------------------------------------------------
0.04742912574947249
Epoch  3
----------------------------------------------------------------------------------
0.036937671164531
Epoch  4
----------------------------------------------------------------------------------
0.031243254078603105
Epoch  5
----------------------------------------------------------------------------------
0.02510889248996432
Epoch  6
----------------------------------------------------------------------------------
0.02301007009422604
Epoch  7
----------------------------------------------------------------------------------
0.020071982094066174
Epoch  8
----------------------------------------------------------------------------------
0.025563417584688882
Epoch  9
----------------------------------------------------------------------------------
0.020532808